<a href="https://colab.research.google.com/github/sleiph/exploracao-de-dados/blob/main/estat%C3%ADstica_agrupados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estatística aplicada pra dados agrupados

In [18]:
# o importante é importar
import numpy as np
import pandas as pd

import math

In [19]:
# quantidade de casas depois da vírgula
precisao = 4

## Declaração das funções

### Desagrupando dados

In [20]:
def DesagruparDados(limites):
  dados = []

  for grupo in limites:
    for i in range(grupo[2]):
      dados.append( (grupo[0]+grupo[1])/2 )

  return dados

### Distribuição de frequência

In [21]:
def DistrubuicaoFreqDF(limites, prec=4):
  #populando a lista

  n = len(limites)
  for i in range(len(limites)):
    limites[i].append( str(limites[i][0]) + " ⊢ " + str(limites[i][1]) )
    limites[i].append( (limites[i][0] + limites[i][1])/2 )

  # criando a tabela
  df = pd.DataFrame(
      limites,
      columns = [ 'inf', 'sup', 'fi', 'dist', 'xi' ]
  )

  # começando o índice em 1
  df.index = np.arange(1, len(df)+1)

  # adicionando valores pra cálculos
  df['fri'] = (df['fi'] / df['fi'].sum()).round(prec)
  df['fri %'] = df['fri'] * 100
  df['fac'] = df['fi'].cumsum()
  df['frac'] = (df['fac'] / df['fi'].sum()).round(prec)
  df['frac %'] = df['frac'] * 100
  df['xf'] = df['xi'] * df['fi']
  df['(xi*xi)*fi'] = (df['xi'] * df['xi']) * df['fi']

  return df

### Medidas de posição

In [22]:
# Média Aritmética
def GetMedia(df, prec=4):
  return round(df['xf'].sum() / df['fi'].sum(), prec)

In [23]:
# Moda
def GetModa(df, prec=4):
  indice_modal = df['fi'].idxmax()
  classe_modal = df.iloc[indice_modal-1]
  frequencia_modal = classe_modal['fi']

  # d1 e d2
  if indice_modal > 1:
    d1 = frequencia_modal - df['fi'].iloc[indice_modal-2]
  else:
    d1 = frequencia_modal
  if indice_modal < len(df):
    d2 = frequencia_modal - df['fi'].iloc[indice_modal]
  else:
    d2 = frequencia_modal

  variacao_classe = df['sup'].min() - df['inf'].min()

  return round(classe_modal['inf'] + (d1/(d1+d2)) * variacao_classe, prec)

#### Percentis

In [24]:
def GetClasse(frequencia, indice):
  contador = 0
  for fac in frequencia:
    if fac >= indice:
      break
    contador += 1
  return contador

In [25]:
def GetPercentil(df, posicao=.5, prec=4):
  frequencia_total = df['fi'].sum()
  indice = frequencia_total * posicao
  classe_indice = GetClasse(df['fac'], indice)
  classe = df.iloc[classe_indice]
  
  limite_inferior = classe['inf']

  frequencia = classe['fi']
  frequencia_anterior = df['fac'].iloc[classe_indice-1]

  variacao_classe = df['sup'].min() - df['inf'].min()

  return round(limite_inferior + ((indice - frequencia_anterior) / frequencia) * variacao_classe, prec)

### Medidas de dispersão

In [26]:
# variância (s²)
def GetVariancia(df, prec=4):
  n = df['fi'].sum()
  return  round( ((1/n) * (df['(xi*xi)*fi'].sum() - ((df['xf'].sum() * df['xf'].sum()) / n )) ), prec)

In [27]:
# desvio padrão (s)
def GetDesvio(df, prec=4):
  return round(math.sqrt( GetVariancia(df) ), prec)

In [28]:
# coeficiente de variação
def GetVariacaoCoef(df, prec=4):
  return round((GetDesvio(df) / GetMedia(df) * 100), prec)

In [29]:
# coeficiente de assimetria de Pearson
def GetAssimetriaCoef(df, prec=4):
  return round( ( 3*(GetMedia(df)-GetPercentil(df, .5)) ) / GetDesvio(df), prec )

In [30]:
# tipo de assimetria
def GetTipoAssimetria(assimetria):
  if assimetria<0:
    return "Assimetria negativa ou à esquerda"
  elif assimetria>0:
    return "Assimetria positiva ou à direita"
  else:
    return "Assimetria nula"

#### curtose

In [31]:
# curtose
def GetCurtose(df, prec=4):
  q1 = GetPercentil(df, .25)
  q3 = GetPercentil(df, .75)
  p90 = GetPercentil(df, .9)
  p10 = GetPercentil(df, .1)
  return round( (q3-q1)/(2*(p90-p10)), prec )

In [32]:
# tipo de curva
def GetTipoCurva(curtose):
  if curtose < 0.263:
    return "Leptocúrtica"
  elif curtose > 0.263:
    return "Platicúrtica"
  else:
    return "Mesocúrtica"

## Entrada de dados

In [33]:
# formato de cada classe: [limite inferior,limite superior, frequência]
dados_agrupados = [
                   [0,2, 10], [2,4, 20], [4,6, 20], [6,8, 40], [8,10, 10]
                  ]

## Distribuição de frequência

In [ ]:
# preparar dados não agrupados
dados_nao_agrupados = DesagruparDados(dados_agrupados)
dados_df = DistrubuicaoFreqDF(dados_agrupados, precisao)

dados_df

## Medidas de posição

In [ ]:
# executando os cálculos
media = GetMedia(dados_df, precisao)
print("media:", media)

In [ ]:
moda = GetModa(dados_df)
print("moda:", moda)

In [ ]:
mediana = GetPercentil(dados_df, .5, precisao)
print("mediana:", mediana)

In [ ]:
q1 = GetPercentil(dados_df, .25, precisao)
print("Q1:", q1)

In [ ]:
q3 = GetPercentil(dados_df, .75, precisao)
print("Q3:", q3)

In [ ]:
p10 = GetPercentil(dados_df, .1, precisao)
print("P10:", p10)

In [ ]:
p90 = GetPercentil(dados_df, .9, precisao)
print("P90:", p90)

## Medidas de dispersão

In [ ]:
variancia = GetVariancia(dados_df, precisao)
print("variância:", variancia)

In [ ]:
desvio = GetDesvio(dados_df, precisao)
print("desvio padrão:", desvio)

In [ ]:
coeficiente_variacao = GetVariacaoCoef(dados_df, precisao)
print("coeficiente de variação:", coeficiente_variacao)

In [ ]:
coeficiente_assimetria = GetAssimetriaCoef(dados_df, precisao)
print("coeficiente de assimetria:", coeficiente_assimetria)

In [ ]:
tipo_assimetria = GetTipoAssimetria(coeficiente_assimetria)
print("tipo de assimetria:", tipo_assimetria)

In [ ]:
curtose = GetCurtose(dados_df, precisao)
print("curtose:", curtose)

In [ ]:
tipo_curva = GetTipoCurva(curtose)
print("tipo de curva:", tipo_curva)